# **TP2 - Exercicio 1 - Grupo 11**

Bruno Miguel Fernandes Araújo - A97509

Tiago Emanuel Lemos Teixeira - A97666

## Implementação em Python

### `declare`
A partir do argumento de entrada **i**, cria um dicionário **state** cujos elementos são formados por: 
- uma chave **cm,n** que vão caracterizar a posição da célula (m,n);
- um valor inteiro (0 ou 1), que vai servir para distinguir se a célula está viva ou morta.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random 

def declare(i):
    state = {}
    for m in range(N+1):
        for n in range(N+1):
            state[f'c{m},{n}']=Symbol(f'c{m},{n}'+str(i),INT)
    return state



### `init`

Recebe a matriz de células "**state**" que por sua vez vai ser preenchida em cada célula por 1 ou 0, de acordo com as seguintes condições:

$$(i = 0 \vee j = 0 \rightarrow c(i,j)\in Borda \wedge c(i,j) = random([0,1],50\% \text{ de chance}))$$ 
$$\vee$$ 
$$(i > 0 \wedge i \le 3 \wedge j > 0 \wedge j \le 3 \rightarrow c(i,j) \in Quadrado(3 \times 3) \wedge c(i,j) = 1)$$
$$\vee$$
$$(c(i,j) = 0)$$

Esta função é só usada para o estado inicial.


In [2]:

def init(state):

    
    res=[]
    
    for i in range(N+1):
        for j in range(N+1):
            
            
            if i==0 or j==0:
                res.append(Equals(state[f'c{i},{j}'], Int(random.randint(0,1)))) #bordas
                
                
            elif i>0 and j<=3 and j>0 and i<=3:
                res.append(Equals(state[f'c{i},{j}'], Int(1))) # 0<i≤3 e 0<j≤3 vivas
                
            else:
                res.append(Equals(state[f'c{i},{j}'], Int(0)))
                       
    #print(res)

    if N>3:
        return And(res)
    
    return

### `trans`

Recebe duas matrizes de células com o mesmo comprimento e largura - **curr** e **prox**. A "**curr**" é um estado já declarado ou provem de uma transição. Assim como a **curr**, a "**prox**" também é um estado já declarado, e é nela que vai ser guardado um novo estado cujos valores dependem do estado **curr** e das condições a seguir mencionadas:

$$\forall_{i,j<N+1}$$
$$(curr[(i,j)] \in Borda \rightarrow curr[(i,j)]=prox[(i,j)])$$
$$\vee$$
$$\text{caso sobreviver}:(curr[(i,j)] = 1 \wedge \text{nº de vizinhos} = 2 \vee \text{nº de vizinhos} = 3 \rightarrow prox[(i,j)] = 1)$$
$$\vee$$
$$\text{caso morrer}:(curr[(i,j)] = 1 \wedge \text{nº de vizinhos} != 2 \wedge \text{nº de vizinhos} != 3 \rightarrow prox[(i,j)] = 0)$$
$$\vee$$
$$\text{caso nascer}: (curr[(i,j)] = 0 \wedge \text{nº de vizinhos} = 3 \rightarrow prox[(i,j)] = 1)$$
$$\vee$$
$$\text{caso contrário}: prox[(i,j)] = 0$$


In [3]:
def trans(curr,prox):
    
    #(i0−i1=±1)∨(j0−j1=±1)
    res = []
    for i in range(N+1):
        for j in range(N+1):
            transit = []
            
            if i == 0 or j == 0: #células da borda
                transit.append(Equals(curr[f'c{i},{j}'],prox[f'c{i},{j}']))
            else:
                if i == N and j == N: # extremo inferior direito
                    t=i-1
                    t_=i
                    l=j-1
                    l_=j
                    
                elif i == N and j>=1 and j!=N: # ultima linha
                    t = i-1
                    t_ = i
                    l = j-1
                    l_ = j+1
                elif j == N and i >=1 and i != N: #ultima coluna
                    t= i-1
                    t_ = i+1
                    l = j-1
                    l_ = j
                    
                else: # células do "miolo" da matriz
                    t=i-1
                    t_=i+1
                    l= j-1
                    l_ = j+1
                

                
                #Se estiver vivo, mantem-se vivo se tiver 2 ou 3 vizinhos vivos,-1 porque o próprio n é vizinho de si mesmo
                transit.append(And(Equals(curr[f'c{i},{j}'],Int(1)),
                                  Or(Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)])-Int(1),Int(3)),
                                    Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)])-Int(1),Int(2))
                                    ),
                                  #Equals(prox[f'c{i},{j}'],curr[f'c{i},{j}'])
                                  Equals(prox[f'c{i},{j}'],Int(1))
                                 ))

                #Se estiver vivo, morre se não tiver 2 ou 3 vizinhos vivos ,-1 porque o próprio n é vizinho de si mesmo
                transit.append(And(Equals(curr[f'c{i},{j}'],Int(1)),
                                  Not(Or(Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)])-Int(1),Int(3)),
                                    Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)])-Int(1),Int(2))
                                    )),
                                  Equals(prox[f'c{i},{j}'],Int(0))
                                 ))

                #Se estiver morto, nasce se tiver 3 vizinhos vivos
                transit.append(And(Equals(curr[f'c{i},{j}'],Int(0)),
                                  Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)]),Int(3)),
                                  Equals(prox[f'c{i},{j}'],Int(1))
                                 ))

                #Se estiver morto,mantém-se morto se não tiver 3 vizinhos vivos
                transit.append(And(Equals(curr[f'c{i},{j}'],Int(0)),
                                  Not(Equals(sum([curr[f'c{m},{n}'] for n in range(l,l_+1) for m in range(t,t_+1)]),Int(3))),
                                 # Equals(prox[f'c{i},{j}'],curr[f'c{i},{j}'])
                                  Equals(prox[f'c{i},{j}'],Int(0))
                                  ))
                
            res.append(Or(transit))    
    return And(res)

### `gera_traco`

Esta função gera um autómato com o nome **states** declarado usando o `declare`, tendo em conta a veracidade das condições geradas pelas funções anteriormente definidas `init` e `trans`.

Cada posição do **states** refere-se a um estado, sendo que o estado inicial é denotado por **states[0]** e restantes dos estados são denotados do **states[1]** ao **states[k-1]** (k é o número total de estados presentes no autómato).

Finalmente, são imprimidos os resultados do **solver**, traduzindo os 0 e 1 de cada passo para ⬜ e ⬛ respetivamente, com o intuito de melhorar a leitura.


**Nota**: só é possivel gerar o autómato se **N** for maior que 3

In [41]:
def gera_traco(k):
        with Solver() as solver:


            # criar k cópias do estado
            states = [declare(i) for i in range(k)]

            # criar o traço
            solver.add_assertion(init(states[0]))
            for i in range(k-1):
                solver.add_assertion(trans(states[i],states[i+1]))


            if solver.solve():
                for i in range(k):
                    print(f">State  {i}:")
                    res=""
                    for m in range(N+1):
                        res+=("\n")
                        for n in range(N+1):
                            #res+=f"{solver.get_value(states[i][f'c{m},{n}'])}"
                            if str(solver.get_value(states[i][f'c{m},{n}']))=="0":
                                res+="⬜"
                            if str(solver.get_value(states[i][f'c{m},{n}']))=="1":
                                res+="⬛"

                    print(res)


            else:
                print("Impossivel criar um caminho")
                
#Exemplo
N=4
gera_traco(10)

>State  0:

⬛⬛⬜⬜⬜
⬜⬛⬛⬛⬜
⬜⬛⬛⬛⬜
⬜⬛⬛⬛⬜
⬛⬜⬜⬜⬜
>State  1:

⬛⬛⬜⬜⬜
⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬛
⬜⬜⬜⬛⬜
⬛⬛⬛⬜⬜
>State  2:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬛⬛⬛⬜
⬛⬛⬛⬜⬜
>State  3:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬜⬛
⬛⬜⬜⬛⬜
>State  4:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬜⬛
⬛⬜⬜⬜⬜
>State  5:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬛⬛
⬛⬜⬜⬜⬜
>State  6:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬛⬛
⬛⬜⬜⬜⬜
>State  7:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬛⬛
⬛⬜⬜⬜⬜
>State  8:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬛⬛
⬛⬜⬜⬜⬜
>State  9:

⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛
⬜⬜⬜⬛⬛
⬛⬜⬜⬜⬜


# **Provas de invariantes**

Nesta parte do trabalho foi nos pedido para verificar os seguintes invariantes:

1. Nunca se alcança o estado de erro;
2. Nenhuma célula normal está permanentemente morta ou permanentemente viva.

# Primeiro invariante

O estado de erro é identificado quando num estado todas as células normais da matriz encontram-se mortas, ou seja, que o valor associado a qualquer célula normal é 0.

A função **bmc_always** recebe um invariante **inv** e um número **K** de estados, e imprime um autómato com o qual o invariante definido pela função **erros** não seja satisfazível.

Se o invariante é satisfeito, então imprime para que número de estados o invariante satisfez (K).

In [7]:
def bmc_always(inv,K):
    
    # K é o tamanho do traço
    # inv = invariante, tem de ser verdadeiro para qualquer estado
        
    with Solver() as solver:
        states = [declare(i) for i in range(K+1)] # declarar estados
        solver.add_assertion(init(states[0])) # satisfazer a inicialização do primeiro estado
        

        for k in range(K):
            if k>0:
                solver.add_assertion(trans(states[k-1],states[k]))

            solver.push()
            solver.add_assertion(Not(inv(states[k]))) # ver se o invariante nao satisfaz no estado k 
           # print(not(inv(states[k])))

            if solver.solve(): # inv nao satisfaz
                print(f">Nonerror case does not holds for {k+1} first states. \nCounter-example:")
                for i,s in enumerate(states[:k+1]):
                    print(f">State  {i}:")
                    res=""
                    for m in range(N+1):
                        res+=("\n")
                        for n in range(N+1):
                            if str(solver.get_value(states[i][f'c{m},{n}']))=="0":
                                res+="⬜"
                            if str(solver.get_value(states[i][f'c{m},{n}']))=="1":
                                res+="⬛"
                    print(res)
                return
            else: #inv satisfaz (assertion not(inv(states[k]))) falhou porque inv satisfez
                if k==K-1:
                    print(f">Nonerror case holds for {K} first states.\n")

            solver.pop()

### `erros`

**state** - matriz de células de um certo estado.

Aqui constrói-se a condição de invariante seguinte:

$$\text{Not}(c(1,1) = 0 \wedge c(1,2) = 0 \wedge \text{...} \wedge c(N,N-1) = 0 \wedge c(N,N) = 0)$$

**Nota**: Células de borda não se contemplam em nenhum dos invariantes.

In [8]:
def erros(state):
    result=[]
    for i in range(1,N+1):
        for j in range(1,N+1):
            result.append(Equals(state[f'c{i},{j}'],Int(0)))
   # print(Not(And(result)))
    return Not(And(result))

#Exemplo
N=10
bmc_always(erros,100)

>Nonerror case holds for 100 first states.



# Segundo invariante

Por fim, o segundo invariante verifica-se se, durante todos os estados, cada uma das células que começam vivas (respetivamente mortas) eventualmente num estado futuro ficam mortas (respetivamente vivas).

A função **bmc_eventually** recebe as funções anteriormente definidas (`declare`,`init` e `trans`), assim como o argumento **prop** que vai ser onde o segundo invariante entra, e o argumento **bound** que indica o número de estado a ser testados.

No que toca ao que devolve, a **bmc_eventually** tem um comportamento análogo ao da **bmc_always**

In [9]:
def bmc_eventually(declare,init,trans,prop,bound):
    
    with Solver() as solver:
        states = [declare(i) for i in range(bound)] # declarar estados
        solver.add_assertion(init(states[0])) # satisfazer a inicialização do primeiro estado
        
        for k in range(bound):
            if k>0:
                solver.add_assertion(trans(states[k-1],states[k]))
                
        solver.add_assertion(Not(prop(states,bound)))

        if solver.solve(): # inv nao satisfaz
            print(f">Invariant does not stand for {bound} states \nCounter-example:")
            for i,s in enumerate(states):
                print(f">State  {i}:")
                res=""
                for m in range(N+1):
                    res+=("\n")
                    for n in range(N+1):
                        if str(solver.get_value(states[i][f'c{m},{n}']))=="0":
                            res+="⬜"
                        if str(solver.get_value(states[i][f'c{m},{n}']))=="1":
                            res+="⬛"
                print(res)
            return
        else: #inv satisfaz (assertion not(inv(states[k]))) falhou porque inv satisfez
            print(f">Nonerror case holds for {bound} first states.\n")
            

### `eventually_liveordead`

**states** - lista de **K** estados

Aqui constrói-se a condição de invariante seguinte:

$$((\sum_{k<K}estado[k][c(1,1)])\ne K \wedge (\sum_{k<K}estado[k][c(1,1)]) \ne 0)$$
$$\wedge$$
$$\text{...}$$
$$\wedge$$
$$((\sum_{k<K}estado[k][c(N,N)])\ne K \wedge (\sum_{k<K}estado[k][c(N,N)]) \ne 0)$$

In [39]:
def eventually_liveordead(states,K):
    list=[]
    for i in range(1,N+1):
        for j in range(1,N+1):
            list.append(And(Not(Equals(sum([states[k][f'c{i},{j}'] for k in range(K)]),Int(K))),
                            Not(Equals(sum([states[k][f'c{i},{j}'] for k in range(K)]),Int(0)))))
    return And(list)

#Exemplo                             
N=4
bmc_eventually(declare,init,trans,eventually_liveordead,10)

>Nonerror case holds for 10 first states.

